In [1]:
from random import Random
import numpy as np
import time
import math
import scipy as sp

import matplotlib.pyplot as plt
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import RBF

In [3]:
#####Setting environment
#================================
def generate_matrix(training_number):
    x_train = np.array(np.linspace(0.01,1.0, training_number), dtype='float32').reshape(training_number, 1)
    rbf= gaussian_process.kernels.RBF()
    Kernel= rbf(x_train, x_train)
    K_y = Kernel + np.eye(training_number) * 1e-8

    x = np.ones((training_number,1), dtype = 'int')

        #Generate y vector 
    y = K_y@x

    #SVD:
    U,S,Vh = np.linalg.svd(K_y)
    c = U.T@y

    #QR:
    Q,R = np.linalg.qr(K_y)
    p = Q.T@y
    return R, p, S, c


In [2]:
def back_substitute(A, b):
    n = b.size
    x = np.zeros(b.shape)
    
    x[n-1] = b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = b[i]
        u = A[i,i+1:n]
        v = x[i+1:n]
        dotproduct = np.dot(u,v)
        x[i] -= dotproduct
        x[i] = x[i]/A[i,i]
    
    return x

def back_substitute2(A, b):
    n = b.size
    x = np.zeros(b.shape)
    
    x[n-1] = b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = b[i]
        for j in range(i+1,n):
            x[i] = x[i] - A[i,j]*x[j]
        
        x[i] = x[i]/A[i,i]
    
    return x

def back_substitute_diag(A, b):
    n = b.size
    b_list = b.ravel()
    x = b_list/A
    
    return x.reshape(n,1)


In [4]:
R, p,S,c = generate_matrix(30)

In [5]:
%timeit back_substitute(R, p)

247 µs ± 51.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit back_substitute2(R, p)

1.79 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%timeit np.linalg.inv(R)@p

52.3 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
%timeit np.linalg.solve(R, p)

25.2 µs ± 766 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%timeit back_substitute_diag(S, c)

1.38 µs ± 17.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit np.linalg.solve(np.diag(S), c)

44 µs ± 8.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
###Check if the results are the same
#===========================================
print(np.allclose(back_substitute(R, p),np.linalg.solve(R, p)))
print(np.allclose(back_substitute2(R, p),np.linalg.solve(R, p)))
print(np.allclose(np.linalg.inv(R)@p,np.linalg.solve(R, p)))
print(np.allclose(back_substitute_diag(S, c),np.linalg.solve(np.diag(S), c)))

True
True
True
True
